In [ ]:
# Library functions

import cv2
import pathlib
import pickle
import joblib
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, Xception, EfficientNetB0, MobileNet
from tensorflow.keras.applications import DenseNet121, NASNetMobile, InceptionV3
from keras import layers, datasets, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.python.keras import regularizers

import os
for dirname, _, filenames in os.walk('/kaggle/input/dataset-41-mri/Dataset_41'):
    print(dirname)
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

In [ ]:
# Function for Mapped Class Labels

def get_label(label):
    if label == 1 :
        return 'Glioma'
    elif label == 2 :
        return 'Meningioma'
    elif label == 3 :
        return 'Pituitary'
    else :
        return 'No Tumor'

# Directory Path

Train_dir = '/kaggle/input/dataset-41-mri/Dataset_41/Training'
Val_dir = '/kaggle/input/dataset-41-mri/Dataset_41/Validation'

In [ ]:
# Data generator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    Train_dir,
    target_size=(128, 128),
    batch_size=100,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    Val_dir,
    target_size=(128, 128),
    batch_size=100,
    class_mode='categorical'
)


In [ ]:
# Display sample images from image sets

for i in range(len(train_generator)//10):
    images, labels = train_generator[i]

# for i in range(len(val_generator)):
#     images, labels = val_generator[i]

    fig=plt.figure(figsize=(15, 15))

    for j in range(len(images)):
        original_image = images[j]
        actual_label = np.array(labels[j]).argmax()

        original_image *= 255

        original_image = original_image.astype('uint8')

        if (j>=0 and j<10):
            fig.add_subplot(1, 10, j%10+1)
            plt.imshow(original_image)
            plt.axis('off')
            plt.title(f"{get_label(actual_label)}")

In [ ]:
# Efficient Net 
# base_model = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(128,128,3))

# ResNet Model
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# DenseNet Model
# base_model = DenseNet121(weights='imagenet',include_top=False,input_shape=(128,128,3))

# NASNet Model
# base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Inception Model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# MobileNet Model
# base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Xception Model
# base_model = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


# apply weights to model
model = base_model.output

# global average for easier computation
model = tf.keras.layers.GlobalAveragePooling2D()(model)

# dropout to avoid overfitting
model = tf.keras.layers.Dropout(rate=0.5)(model)

# output layer
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=base_model.input, outputs = model)

model.summary()

# compile the model
model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics= ['accuracy'])

In [ ]:
# Train the model

history = model.fit(train_generator, epochs=10, validation_data=val_generator)

print(f"\nTraining Accuracy:{history.history['accuracy'][-1] * 100:.2f}%")

In [ ]:
# Visualize training history

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Save the trained model

path='/kaggle/working/'


# Create folders if they don't exist
if not os.path.exists(path):
    os.makedirs(path)

# Save the trained model
model.save(path + 'inception_model.keras')

In [ ]:
# Enter the test directory

Test_dir = '/kaggle/input/dataset-41-mri/Dataset_41/Testing'

# Testing Directory Modification

# Create an ImageDataGenerator for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory
test_generator = test_datagen.flow_from_directory(
    Test_dir,
    target_size=(128, 128),
    batch_size=100,
    class_mode='categorical'
)

In [ ]:
# Import the Model

model = keras.models.load_model('/kaggle/working/inception_model.keras')

# Testing Value_accuracy and Value_Loss

test_loss, test_acc = model.evaluate(test_generator)
print(f"\nTest accuracy: {test_acc * 100:.2f}%")


In [ ]:
# Display predicted images from image sets

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    predictions = model.predict(images)

    fig=plt.figure(figsize=(15, 15))

    for j in range(len(images)):
        original_image = images[j]
        actual_label = np.array(labels[j]).argmax()
        predicted_label = predictions[j].argmax()
        predicted_probability = max(predictions[j])
        
        original_image *= 255
        original_image = original_image.astype('uint8')

        if (j>=0 and j<5):
            fig.add_subplot(1, 5, j%5+1)
            plt.imshow(original_image)
            plt.axis('off')
            plt.title(f"Probability: {predicted_probability:.2f}\n"
                  f"Predict: {get_label(predicted_label)}\n"
                  f"Actual: {get_label(actual_label)}")

In [ ]:
# Display Confusion Matrix

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

all_images = []
all_labels = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    all_images.append(images)
    all_labels.append(labels)
    
# Concatenate the lists to form a single array
all_images = np.concatenate(all_images, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

actual_labels = np.argmax(all_labels, axis = 1)
prediction_labels = np.argmax(model.predict(all_images),axis = 1)

conf_matrix = confusion_matrix(actual_labels, prediction_labels)

# Create label names for the confusion matrix
label_names = [get_label(i) for i in range(4)]

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',xticklabels=label_names, yticklabels=label_names)
plt.xlabel('Predicted Labels', fontsize=12, fontweight='bold')
plt.ylabel('True Labels', fontsize=12, fontweight='bold')
plt.show()


In [ ]:
# Generate classification report
report = classification_report(actual_labels, prediction_labels, target_names=label_names, output_dict=True)

# Convert classification report to DataFrame
report_df = pd.DataFrame(report).transpose().drop(['accuracy', 'macro avg', 'weighted avg'])
# Select only relevant columns
report_df = report_df[['precision', 'recall', 'f1-score']]

# Plot classification report
plt.figure(figsize=(8, 6))
sns.heatmap(report_df.iloc[:, :].T, annot=True, cmap='Blues')
plt.ylabel('Metrics', fontsize=12, fontweight='bold')
plt.xlabel('Classes', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
# Display Graphs

from sklearn.metrics import roc_curve, precision_recall_curve, auc

# Convert labels to binary format
binary_actual_labels = np.where(actual_labels == 0, 0, 1)
binary_prediction_labels = np.where(prediction_labels == 0, 0, 1)


# Receiver Operating Characteristic (ROC) Curve
fpr, tpr, threshold = roc_curve(binary_actual_labels, binary_prediction_labels)

# Compute ROC curve and ROC area
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()

print()

# Precision-Recall (PR) Curve
precision, recall, threshold = precision_recall_curve(binary_actual_labels, binary_prediction_labels)

# Compute PR curve and PR area
pr_auc = auc(recall,precision)

# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.step(recall, precision, color='b', where='post', label=f'PR curve (AUC = {pr_auc:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc='upper right')
plt.show()